A lo largo de esta memoria, se explicará de qué manera se ha resuelto el problema final del módulo de __Introducción a Data Science__. Trataremos el __problema 1__ y el __problema 2__ por separado, y se irán mostrando los resultados de ejecución más relevantes del código propuesto en Python.
Además, en el propio código se incluirán comentarios para facilitar la comprensión de la solución propuesta.

# Problema 1

En primer lugar, importamos las __librerías__ que nos harán falta para resolver el problema:

In [1]:
import json
import pandas as pd

Pediremos como input los ficheros de texto __"Tweets.txt"__ y __"Sentimientos.txt"__, respectivamente. Ambos se encuentran en el mismo directorio que el notebook:

In [2]:
sentimientos = open(input("Introduzca el nombre del archivo de SENTIMIENTOS, con su extensión: ")) 
tweets = open(input("Introduzca el nombre del archivo de TWEETS, con su extensión: ")) 

Introduzca el nombre del archivo de SENTIMIENTOS, con su extensión: Sentimientos.txt
Introduzca el nombre del archivo de TWEETS, con su extensión: Tweets.txt


Almacenamos en un diccionario llamado __"valores"__ la clave y el valor asociados a cada linea del archivo de sentimientos, empleando el fragmento de código facilitado en el enunciado:

In [3]:
valores = {} 
for linea in sentimientos: 
    termino, valor = linea.split("\t") 
    valores[termino] = int(valor) 
print (valores.items() )

dict_items([('abandon', -2), ('abandoned', -2), ('abandons', -2), ('abducted', -2), ('abduction', -2), ('abductions', -2), ('abhor', -3), ('abhorred', -3), ('abhorrent', -3), ('abhors', -3), ('abilities', 2), ('ability', 2), ('aboard', 1), ('absentee', -1), ('absentees', -1), ('absolve', 2), ('absolved', 2), ('absolves', 2), ('absolving', 2), ('absorbed', 1), ('abuse', -3), ('abused', -3), ('abuses', -3), ('abusive', -3), ('accept', 1), ('accepted', 1), ('accepting', 1), ('accepts', 1), ('accident', -2), ('accidental', -2), ('accidentally', -2), ('accidents', -2), ('accomplish', 2), ('accomplished', 2), ('accomplishes', 2), ('accusation', -2), ('accusations', -2), ('accuse', -2), ('accused', -2), ('accuses', -2), ('accusing', -2), ('ache', -2), ('achievable', 1), ('aching', -2), ('acquit', 2), ('acquits', 2), ('acquitted', 2), ('acquitting', 2), ('acrimonious', -3), ('active', 1), ('adequate', 1), ('admire', 3), ('admired', 3), ('admires', 3), ('admiring', 3), ('admit', -1), ('admits',

Al tratar los tweets, más adelante, lo primero que haremos será buscar coincidencias con claves formadas por más de una palabra, y actualizar el texto del tweet quitando esas claves. De esta manera, si una frase es __"You're messing up with me"__, eliminaríamos __"messing up"__ para no contabilizar doblemente la palabra __"messing"__, en el caso de que apareciese como clave formada por una sola palabra. Para seguir tratando el tweet, se usará la función __"analiza_tweet"__, de la que luego hablaremos.

En este ejemplo, las claves del diccionario formadas por dos palabras o más son:

In [4]:
for clave,valor in valores.items():
    if len(clave.split())>1:
        print(clave)

can't stand
cashing in
cool stuff
does not work
dont like
fed up
green wash
green washing
messing up
no fun
not good
not working
right direction
screwed up
some kind


Con respecto al fichero que contiene la información de los tweets en formato __.json__, almacenaremos en una lista llamada __"tweets_info"__ las cadenas __JSON__ que contengan la clave __"text"__, ya que se corresponderán con las cadenas que almacenan tweets:

In [5]:
tweets_info=[]  # Lista que almacenará las lineas del "Tweets.txt" que sean TWITS
tweets_text=[]  # Lista que almacenará el texto de los twits

In [6]:
for line in tweets:
    # Los tweets figurarán en las líneas en las que 'text' exista como clave del diccionario
    if 'text' in json.loads(line).keys(): 
        tweets_info.append(json.loads(line))
        
for tw in pd.DataFrame(tweets_info)['text']: # Añadimos valores a la lista "tweets_text"
    tweets_text.append(tw)
    
(tweets_text) # Mostramos los tweets a la salida 

['@Brenamae_ I WHALE SLAP YOUR FIN AND TELL YOU ONE LAST TIME: GO AWHALE',
 'Metin Şentürk Twitterda @metinsenturk MUHTEŞEM ÜÇLÜ; SEN, BEN, MÜZİK',
 'RT @byunghns: 😭 I LOVE #틴탑 SO MUCH #쉽지않아 IS GOING TO BE SO GOOD 😭',
 'que hdp maicon lo que le hizo a david luiz jajajajajajajajajajaj,igual se jodio la carrera',
 'ドライ！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！',
 'RT @Positivamos: tudo o que eu quero é um amor sincero.',
 'RT @GossipRoomOff: Si Nathalie sait pertinemment que son couple ne va pas durer, alors pourquoi accepter la demande en mariage de Vivian ? …',
 'RT @Dayannalozano_: Junior va perdiendo... http://t.co/5mOsR7boLx',
 'RT @Positivamos: tudo o que eu quero é um amor sincero.',
 'RT @Positivamos: tudo o que eu quero é um amor sincero.',
 'الله أكبر الله أكبر الله أكبر, لا إله إلا الله, الله أكبر الله اكبر ولله الحمد http://t.co/M454OhQpYY',
 "TWİTTER'I AKRABALAR KEŞFEDİCEK DİYE ÇOK KORKUYORUM \n Metin Şentürk Twitterda @metinsenturk",
 'RT @PINKBATHARRY: les saintes nitouches de ma

Ahora hay que analizar los tweets conforme se especifica en el enunciado del problema. Deberemos encontrar correspondencias entre las palabras que se encuentran en cada tweet y las palabras que figuran como clave en el diccionario __"valores"__. Como no se trata de una tarea trivial, vamos a presentar las funciones que se han definido para tratar los tweets, teniéndose en cuenta varias consideraciones.

In [7]:
def separa_oracion(cadena): # Devuelve una lista en la que cada componente es una lista de las palabras que conforman una 
    resultado=[]            # sola de las oraciones del tweet. Las oraciones se separan por caracteres no alfanuméricos.
    oracion=[]              # Esta función tiene interés para analizar palabras formadas por letras separadas en una
                            # misma oración. 
        
    forbidden=["'t","'s","’t","’s","'m","’m","'re","’re"]   # Referencia a cualquier palabra contraída
        
    for palabra in cadena:
        if palabra.isalpha() or any(f in palabra.lower() for f in forbidden): # No separaremos las palabras contraídas
            oracion.append(palabra)
        # Para cada palabra con un caracter no alfabético...
        elif (not palabra.isalpha() and '#' not in palabra): # (No consideramos los hashtags como palabras)                                                 
            pal=''                                             
            for letra in palabra:
                if(letra.isalpha()):
                    pal+=letra
                elif not letra.isalpha():
                    oracion.append(pal)
                    resultado.append(oracion)
                    oracion=[]
                    pal=''
            if (pal.isalpha() and pal!=''):
                oracion.append(pal)
                
    resultado.append(oracion)
    return resultado

In [8]:
c='She was my best friend .At last, I felt trully H A P P Y.I had soooo much fuuuuun. Best experience ever'
separa_oracion(c.split())

[['She', 'was', 'my', 'best', 'friend', ''],
 ['At', 'last'],
 ['I', 'felt', 'trully', 'H', 'A', 'P', 'P', 'Y'],
 ['I', 'had', 'soooo', 'much', 'fuuuuun'],
 ['Best', 'experience', 'ever']]

In [9]:
def letra_repetida(palabra): # Si hay palabras con una letra repetida un gran número de veces, devolvemos la palabra intacta y 
    letra_repetida=False     # la palabra sin el caracter alargado en caso de coincidencia
    resultado=[]              
    palabra_final=''
    letra_anterior=''
    for letra in palabra:
        if(letra==letra_anterior): # Si se repite una letra
            if(letra_repetida==False):
                letra_repetida=True
        else:
            palabra_final+=letra
            letra_repetida=False
        letra_anterior=letra
    resultado.append(palabra)
    resultado.append(palabra_final)
    return resultado
    

In [10]:
letra_repetida('loveeeee')

['loveeeee', 'love']

In [11]:
letra_repetida('aaaawesome')

['aaaawesome', 'awesome']

In [12]:
letra_repetida('iiincreeeeiiiibleeeeeee')

['iiincreeeeiiiibleeeeeee', 'increible']

In [13]:
def palabra_separada(cadena): # Tomando la cadena en la que está almacenado el tweet, vuelca en el resultado las palabras
    separada=''               # que estén escritas separadas caracter a caracter, en caso de que sea una palabra que se 
    contador=0                # encuentre en el diccionario. Considera el caso en que haya varias palabras escrita así en el
    resultado=[]              # tweet.
    for palabra in cadena:
        if(len(palabra)==1):
            separada+=palabra
            contador+=1
        else:
            if(contador>1):
                contador=0
                resultado.append(separada)
            separada=''
    resultado.append(separada)
    return resultado

In [14]:
c='He was like a real g o d and the show was m a g i c a l'
palabra_separada(c.split())

['god', 'magical']

Hemos llamado __"analiza_oracion"__ a la función que busca en el diccionario __"valores"__ las palabras que forman parte de la cadena de entrada. 

In [15]:
def analiza_oracion(cadena):
    resultado=0
    for palabra in cadena:
        if palabra in valores.keys():
            resultado+=valores[palabra]
    return resultado

In [16]:
valores['happy']

3

In [17]:
valores['best']

3

In [18]:
c='She was my best friend and she made me so happy'
analiza_oracion(c.split()) # Devuelve el sentimiento total de la cadena

6

A continuación presentaremos la función __"analiza_tweet"__ para considerar todos los casos explicados anteriormente.

__Nótese que es aquí cuando convertimos a minúsculas las palabras para buscarlas en el diccionario.__

In [19]:
def analiza_tweet(tweet):
    cadena_total=[]
    oraciones=separa_oracion(tweet.split())
    
    for c in oraciones:
        cadena=c+palabra_separada(c)
        for palabra_it in cadena:
            palabra=palabra_it.lower() # Ya que las palabras del diccionario aparecen en minúsculas
            cadena_total.extend(list(set(letra_repetida(palabra))))
    
    return analiza_oracion(cadena_total)
    

Se puede apreciar que, al usar __"set"__, no se considera la misma palabra dos veces en la lista que toma como argumento. En la cadena total se contabilizan todas las palabras para analizar el sentimiento total del tweet, en caso de que se repitan palabras en distintas frases. 

Aquí aplicamos __"analiza_oracion"__:

In [20]:
c="She was my best friend. At last, I felt trully H A P P Y. I can't STAND all this fuuuuun. Best experience ever"
sentimiento_total=0
c=c.lower() # Por si aparece en mayúscula alguna clave

for clave,valor in valores.items(): # Para tener en cuenta las claves del diccionario "valores" con más de una palabra
    if len(clave.split())>1 and clave in c:
        sentimiento_total+=valor
        c=c.replace(clave,' ')

sentimiento_total+=analiza_tweet(c) 
sentimiento_total

10

__Realizamos la comprobación manual__:

In [21]:
ch=['best','happy','fun','best',"can't stand"] # Palabras del tweet incluidas en el diccionario "valores"
analiza_oracion(ch)
    

10

## Resultados Ejercicio 1

En lo referente a la salida del programa, obtenemos:

In [22]:
for tweet in tweets_text: # Para cada tweet de la lista
    sentimiento_total=0
    tweet_original=tweet # Guardamos el tweet sin extraer palabras de él, para imprimirlo por pantalla
    tweet=tweet.lower()

    for clave,valor in valores.items(): # Para tener en cuenta las claves del diccionario "valores" con más de una palabra
        if len(clave.split())>1 and clave in tweet:
            sentimiento_total+=valor
            tweet=tweet.replace(clave,'')

    sentimiento_total+=analiza_tweet(tweet) 
    
    print("EL SIGUIENTE TWEET: '",str(tweet_original),"' TIENE UN SENTIMIENTO ASOCIADO DE:",sentimiento_total)

EL SIGUIENTE TWEET: ' @Brenamae_ I WHALE SLAP YOUR FIN AND TELL YOU ONE LAST TIME: GO AWHALE ' TIENE UN SENTIMIENTO ASOCIADO DE: 0
EL SIGUIENTE TWEET: ' Metin Şentürk Twitterda @metinsenturk MUHTEŞEM ÜÇLÜ; SEN, BEN, MÜZİK ' TIENE UN SENTIMIENTO ASOCIADO DE: 0
EL SIGUIENTE TWEET: ' RT @byunghns: 😭 I LOVE #틴탑 SO MUCH #쉽지않아 IS GOING TO BE SO GOOD 😭 ' TIENE UN SENTIMIENTO ASOCIADO DE: 7
EL SIGUIENTE TWEET: ' que hdp maicon lo que le hizo a david luiz jajajajajajajajajajaj,igual se jodio la carrera ' TIENE UN SENTIMIENTO ASOCIADO DE: 0
EL SIGUIENTE TWEET: ' ドライ！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！ ' TIENE UN SENTIMIENTO ASOCIADO DE: 0
EL SIGUIENTE TWEET: ' RT @Positivamos: tudo o que eu quero é um amor sincero. ' TIENE UN SENTIMIENTO ASOCIADO DE: 0
EL SIGUIENTE TWEET: ' RT @GossipRoomOff: Si Nathalie sait pertinemment que son couple ne va pas durer, alors pourquoi accepter la demande en mariage de Vivian ? … ' TIENE UN SENTIMIENTO ASOCIADO DE: 0
EL SIGUIENTE TWEET: ' RT @Dayannalozano_: Junio

EL SIGUIENTE TWEET: ' Metin Şentürk Twitterda @metinsenturk Sen o kızı üzüyorsun ama o seni seviyor. ' TIENE UN SENTIMIENTO ASOCIADO DE: 0
EL SIGUIENTE TWEET: ' @tweetpotweet @RTS_Sport Bruka!!! ' TIENE UN SENTIMIENTO ASOCIADO DE: 0
EL SIGUIENTE TWEET: ' عمره ١١ سنة ويكتب : ترى الجفاء بعد الوفاء عندنا عيب . 

اي جفاء واي وفاء ؟ اربع اغاني لام كلثوم 
اطول من حياتك "̮*beer* . ' TIENE UN SENTIMIENTO ASOCIADO DE: 0
EL SIGUIENTE TWEET: ' RT @mmez11: للمتواجدين ⭕لمتابعة أكثر من[900 الف]مغرد ➊ فولو مي @mmez11 ➋ ريتويت ➌ فولو للي عمل رتويت ➍ فولو باك ➎ اللي ما يلتزم ما بيستفيد … ' TIENE UN SENTIMIENTO ASOCIADO DE: 0
EL SIGUIENTE TWEET: ' やったー続編！ ' TIENE UN SENTIMIENTO ASOCIADO DE: 0
EL SIGUIENTE TWEET: ' @4qh3_ 1o3q8ieshi ' TIENE UN SENTIMIENTO ASOCIADO DE: 0
EL SIGUIENTE TWEET: ' @mariam_63  entra aca y le ragalas UNOs 15 minutos de Votos a Lucero Xfa. Gracias !!! http://t.co/CLTKgmdxE8 #YSEA ' TIENE UN SENTIMIENTO ASOCIADO DE: 0
EL SIGUIENTE TWEET: ' Sneezing like hell ' TIENE UN SENTIMIENTO

EL SIGUIENTE TWEET: ' Paint the town this weekend at @supercrawl! It’s easy to get to there with GO. http://t.co/2Zz0z72zDz #Supercrawl2014 http://t.co/rsQpACFL77 ' TIENE UN SENTIMIENTO ASOCIADO DE: 1
EL SIGUIENTE TWEET: ' 昨日ストーカーに殺されかける夢見たんだけど

襲われてると殺されるの中間だからまあまあかな… 殺される夢は吉夢なんだね… http://t.co/WNvN8EdSUP ' TIENE UN SENTIMIENTO ASOCIADO DE: 0
EL SIGUIENTE TWEET: ' Thank god for the GR skywalk!! #lakersthatdontwanttogetwet ' TIENE UN SENTIMIENTO ASOCIADO DE: 3
EL SIGUIENTE TWEET: ' Best person in the world.💕🌎🌻👯 http://t.co/nwuiiHLU07 ' TIENE UN SENTIMIENTO ASOCIADO DE: 3
EL SIGUIENTE TWEET: ' Lost my headphones , my day is over 😤😤 ' TIENE UN SENTIMIENTO ASOCIADO DE: -3
EL SIGUIENTE TWEET: ' People at my school think being gay is a choice but if being gay really is a choice why do most gay people including me want to die? pt.1 ' TIENE UN SENTIMIENTO ASOCIADO DE: -2
EL SIGUIENTE TWEET: ' @Peeweezzy exactly why a move would be considered a serious option. I'm going to the fútbol stadium no

EL SIGUIENTE TWEET: ' RT @akatyan2: 突然大人の表情になる赤ちゃん

https://t.co/CN4jx9yKIh ' TIENE UN SENTIMIENTO ASOCIADO DE: 0
EL SIGUIENTE TWEET: ' RT @sandratorensma: http://t.co/Np9Z6LN95y ' TIENE UN SENTIMIENTO ASOCIADO DE: 0
EL SIGUIENTE TWEET: ' RT @Mo_nrm3: تاخذ من أوصاف الغزال التلفات 
وتأخذ من الصقر المبرقع عيونه

 من زينها لو يجمعون الملذات 
يبطون وصف الفاتنه ياصلونه" http://t.c… ' TIENE UN SENTIMIENTO ASOCIADO DE: 0
EL SIGUIENTE TWEET: ' Destiny初日で売り上げ5億ドルかよ凄いな。 ' TIENE UN SENTIMIENTO ASOCIADO DE: 0
EL SIGUIENTE TWEET: ' Recién veo ese mensaje! ' TIENE UN SENTIMIENTO ASOCIADO DE: 0
EL SIGUIENTE TWEET: ' @nialls_lovexz @aMeezys_wiFey_ murgaa LOL
hahaha ' TIENE UN SENTIMIENTO ASOCIADO DE: 6
EL SIGUIENTE TWEET: ' RT @sergioaruizh: Lo malo de las mentes cerradas, es que vienen con bocas abiertas. ' TIENE UN SENTIMIENTO ASOCIADO DE: 0
EL SIGUIENTE TWEET: ' 132 ' TIENE UN SENTIMIENTO ASOCIADO DE: 0
EL SIGUIENTE TWEET: ' @NourhanIsmail97 MH we lolo we 2 mesh hat3rfyhom ' TIENE UN SENTIMIENTO AS

# Problema 2

Tenemos todos los tweets almacenados en la lista __"tweets_text"__, por lo que consideraremos las palabras que no aparecen como clave en el diccionario __"valores"__, el cual ya obtuvimos en el problema anterior.

__Como criterio para asignar sentimientos a palabras que no estén en el diccionario, se tomará el sentimiento total del tweet al que pertenece la palabra.__

En el primer problema definimos funciones para identificar palabras que podrían estar "mal escritas" para asignar el sentimiento del tweet. En este segundo apartado, se considerarán las palabras que conforman el tweet intentando tener en cuenta esos casos. Se tendrá en cuenta lo siquiente:
* No consideraremos los "RT" ni las referencias a personas o grupos como palabras (aquellas con el caracter "@").
* No se considerarán los enlaces web ("http").
* Se imprimirán por pantalla las palabras sin caracteres no alfanuméricos ("free" en lugar de "free,", por ejemplo).
* No se han tenido en cuenta los hashtags (palabras con el caracter "#")
* Se recurrirá a la función __"letra_repetida"__ para descartar las palabras que estén en __"valores"__. Por ejemplo, si la palabra __"loveeee"__ aparece en un tweet, la detecta como __"love"__ y no la considerará en el resultado.
* La palabras contraídas (aquellas con caracteres __'s__ o __'t__, por ejemplo) no se separarán y se mostrarán a la salida del programa tal como aparecen. La función que gestiona esto es __"separa_oracion"__. Por ejemplo, __"don't"__ se considera como palabra única a la salida. 
* Se recurrirá a la función __"quita_palabra_separada"__ para descartar las palabras que estén en __"valores"__. Se explica a continuación:


In [23]:
def quita_palabras_separadas(cadena):
    cadena_final=cadena
    palabras=palabra_separada(cadena_final.split()) # Se obtiene una lista con las palabras separadas
    for palabra in palabras:
        p=''
        # Compruebo si la palabra aparece en el diccionario. Si es así, modifico la cadena quitando esa palabra
        if palabra!='' and (palabra.lower() in valores.keys()):
            for letra in palabra:
                p=p+letra+' '
            p=p[:-1]
            cadena_final=cadena_final.replace(p,' ')
            
     
    # Devuevo la cadena modificada o intacta, según se hayan encontrado palabras separadas en el diccionario.
    if ('' in palabras) and len(palabras)==1:
        return cadena
    else:
        return cadena_final
    

A continuación se muestran varios ejemplos para ver su utilidad. 
* El primer tweet figura en el fichero __"Tweets.txt"__, y se comprueba que elimina las letras que forman la palabra del diccionario.
* En el segundo ejemplo se ve como solo quita la palabra __"god"__ de la cadena, porque __"magical"__ no aparece en el diccionario.
* En la tercera frase de ejemplo, se aprecia cómo deja la cadena intacta al no tener una palabra separada del diccionario.

In [24]:
quita_palabras_separadas('Just waking up in the morning gotta thank G O D ❤️❤️❤️❤️❤️🙏🙏')

'Just waking up in the morning gotta thank   ❤️❤️❤️❤️❤️🙏🙏'

In [25]:
quita_palabras_separadas('He was like a real g o d and m a g i c a l.')

'He was like a real   and m a g i c a l.'

In [26]:
quita_palabras_separadas('Esto es una frase corriente')

'Esto es una frase corriente'

## Resultados Ejercicio 2

Con la siguiente sección de código, se imprime por pantalla las palabras de cada tweet que no aparecen en el diccionario __"valores"__, y se les asigna el sentimiento total del tweet en el que aparecen.

In [27]:
forb_char=['http','@','RT','#'] # No se consideran palabras que contengan estos caracteres
for tweet in tweets_text: # Analizo cada tweet de la lista
    
    # Imprimo el tweet intacto por pantalla
    print("TWEET: '",tweet,"'")
    
    # Modifico el tweet quitando las claves del diccionario de más de una palabra
    sentimiento_total=0
    for clave,valor in valores.items(): 
        if len(clave.split())>1 and clave in tweet:
            sentimiento_total+=valor
            tweet=tweet.replace(clave,'')
    sentimiento_total+=analiza_tweet(tweet) # Sentimiento del tweet considerado
    
    
    # Modifico el tweet, quitando las palabras separadas del diccionario si las hay.
    tweet=(quita_palabras_separadas(tweet)) 
    
    tweet_list=tweet.split() # Lista con las palabras del tweet
    tweet_list_cpy=tweet_list.copy() # Copia del tweet, que voy modificando conforme recorro la lista original
    
    # Elimino las palabras con coincidencias con elementos de "forb_char"
    for palabra in tweet_list:
        if any(fb in palabra for fb in forb_char):
            tweet_list_cpy.remove(palabra)
    
    oraciones=separa_oracion(tweet_list_cpy) # Separo en oraciones el tweet considerado
    
    for o in oraciones: # Analizo cada oración del tweet
        for palabra in o: # Trato las palabras que conforman el tweet
            
            # Considero tambien las palabras con letras repetidas que estén en el diccionario (como "loveeee")
            condic=(palabra.lower() not in valores.keys()) and (letra_repetida(palabra)[-1].lower() not in valores.keys())
            if (palabra!='') and condic:                # Imprimo las palabras no vacías que no figuran en el 
                print(palabra,':',sentimiento_total)    # diccionario "valores"
    print("\n")
        

TWEET: ' @Brenamae_ I WHALE SLAP YOUR FIN AND TELL YOU ONE LAST TIME: GO AWHALE '
I : 0
WHALE : 0
SLAP : 0
YOUR : 0
FIN : 0
AND : 0
TELL : 0
YOU : 0
ONE : 0
LAST : 0
TIME : 0
GO : 0
AWHALE : 0


TWEET: ' Metin Şentürk Twitterda @metinsenturk MUHTEŞEM ÜÇLÜ; SEN, BEN, MÜZİK '
Metin : 0
Şentürk : 0
Twitterda : 0
MUHTEŞEM : 0
ÜÇLÜ : 0
SEN : 0
BEN : 0
MÜZİK : 0


TWEET: ' RT @byunghns: 😭 I LOVE #틴탑 SO MUCH #쉽지않아 IS GOING TO BE SO GOOD 😭 '
I : 7
SO : 7
MUCH : 7
IS : 7
GOING : 7
TO : 7
BE : 7
SO : 7


TWEET: ' que hdp maicon lo que le hizo a david luiz jajajajajajajajajajaj,igual se jodio la carrera '
que : 0
hdp : 0
maicon : 0
lo : 0
que : 0
le : 0
hizo : 0
a : 0
david : 0
luiz : 0
jajajajajajajajajajaj : 0
igual : 0
se : 0
jodio : 0
la : 0
carrera : 0


TWEET: ' ドライ！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！ '
ドライ : 0


TWEET: ' RT @Positivamos: tudo o que eu quero é um amor sincero. '
tudo : 0
o : 0
que : 0
eu : 0
quero : 0
é : 0
um : 0
amor : 0
sincero : 0


TWEET: ' RT @GossipRoomOff: Si Nathali

TWEET: ' RT @Positivamos: tudo o que eu quero é um amor sincero. '
tudo : 0
o : 0
que : 0
eu : 0
quero : 0
é : 0
um : 0
amor : 0
sincero : 0


TWEET: ' @Aurinth I think it's possible, and sometimes scenes like that can work as long as they're bookended by some sort of development. '
I : 2
think : 2
it's : 2
possible : 2
and : 2
sometimes : 2
scenes : 2
that : 2
can : 2
work : 2
as : 2
long : 2
as : 2
they're : 2
bookended : 2
by : 2
some : 2
sort : 2
of : 2
development : 2


TWEET: ' http://t.co/mFdtyT81ux '


TWEET: ' RT @Positivamos: tudo o que eu quero é um amor sincero. '
tudo : 0
o : 0
que : 0
eu : 0
quero : 0
é : 0
um : 0
amor : 0
sincero : 0


TWEET: ' RT @Positivamos: tudo o que eu quero é um amor sincero. '
tudo : 0
o : 0
que : 0
eu : 0
quero : 0
é : 0
um : 0
amor : 0
sincero : 0


TWEET: ' @elofwind 중소기업이라 잡무만 따로 뽑지는 않지만 면접때 막내는 이런것도 해야한다 분명히 말을 하고 뽑아도 그런말 들어본 적 없다는듯이 행동하는 애들이 점점 많아집니다.. -.- @misty8628 '
중소기업이라 : 0
잡무만 : 0
따로 : 0
뽑지는 : 0
않지만 : 0
면접때 : 0
막내는 : 0
이런것도 : 0
해야한다 

فولو : 0
مي : 0
ريتويت : 0
فولو : 0
للي : 0
عمل : 0
رتويت : 0
فولو : 0
باك : 0
اللي : 0
ما : 0
يلتزم : 0
ما : 0
بيستفيد : 0


TWEET: ' やったー続編！ '
やったー続編 : 0


TWEET: ' @4qh3_ 1o3q8ieshi '
o : 0
q : 0
ieshi : 0


TWEET: ' @mariam_63  entra aca y le ragalas UNOs 15 minutos de Votos a Lucero Xfa. Gracias !!! http://t.co/CLTKgmdxE8 #YSEA '
entra : 0
aca : 0
y : 0
le : 0
ragalas : 0
UNOs : 0
minutos : 0
de : 0
Votos : 0
a : 0
Lucero : 0
Xfa : 0
Gracias : 0


TWEET: ' Sneezing like hell '
Sneezing : -2


TWEET: ' @Belu_Cervantes jajaja.. felicidaaaaadeees tontula me acabo de enterar que aprobaste '
jajaja : 0
felicidaaaaadeees : 0
tontula : 0
me : 0
acabo : 0
de : 0
enterar : 0
que : 0
aprobaste : 0


TWEET: ' ஙே என்னாச்சி பழசு வது ஙே '
ங : 0
என : 0
ன : 0
ச : 0
ச : 0
பழச : 0
வத : 0
ங : 0


TWEET: ' En el bus demos prioridad a niños,mujeres embarazadas, personas de la tercera edad y con discapacidad. Hagamos de Montería una #CiudadAmable '
En : 0
el : 0
bus : 0
demos : 0
prioridad : 0
a : 0
ni

QUIERES : 0
ESTA : 0
POR : 0
TU : 0
CUMPLEAÑOS : 0


TWEET: ' @kashiwaren0818 暇な日あそぼ！ '
暇な日あそぼ : 0


TWEET: ' @kirisoor おやすみなさい〜( ˘ω˘ ) '
おやすみなさい : 0
ω : 0


TWEET: ' Buenas buenas buenas buenas buenas buenas buenas '
Buenas : 0
buenas : 0
buenas : 0
buenas : 0
buenas : 0
buenas : 0
buenas : 0


TWEET: ' やったね3期うおおおおおおおおおおおおおおおおおおおおおおおおお #priya #prismaillya #2wei #tokyomx '
やったね : 0
期うおおおおおおおおおおおおおおおおおおおおおおおおお : 0


TWEET: ' RT @Max_Maillots: Ils ont mi la célébration de Eto'o dans #FIFA15, la mort !!
http://t.co/gjKYCSZ4xK '
Ils : 0
ont : 0
mi : 0
la : 0
célébration : 0
de : 0
Eto : 0
o : 0
dans : 0
la : 0
mort : 0


TWEET: ' @H109Z بحاول يعني D': '
بحاول : 0
يعني : 0
D : 0


TWEET: ' RT @ChelseaBorrelli: Take me back to vaca http://t.co/QR9viwrGpn '
Take : 0
me : 0
back : 0
to : 0
vaca : 0


TWEET: ' @yomaira98x dude u have no idea how funny it was 😂 '
dude : 3
u : 3
have : 3
idea : 3
how : 3
it : 3


TWEET: ' Tidakkk rosakk speaker phone!! Huaaa!!! '
Tidakkk : 0
rosakk : 0
speaker : 

gracioso : 0
cuando : 0
aquello : 0
que : 0
ayer : 0
te : 0
dolía : 0
en : 0
el : 0
alma : 0
hoy : 0
sólo : 0
pasa : 0
a : 0
ser : 0
un : 0
simple : 0
recuerdo : 0


TWEET: ' @anyii_1412 jajajjajaj se parece a vos en todas las grabaciones que tengo '
jajajjajaj : 0
se : 0
parece : 0
a : 0
vos : 0
en : 0
todas : 0
las : 0
grabaciones : 0
que : 0
tengo : 0


TWEET: ' RT @Oso_GlodUp: Lol That Boy Young Thug Snapped On Kevin Gates , Them Niggas Gone Kill Each Other ! '
That : -5
Boy : -5
Young : -5
Thug : -5
Snapped : -5
On : -5
Kevin : -5
Gates : -5
Them : -5
Gone : -5
Each : -5
Other : -5


TWEET: ' Metin Şentürk Twitterda @metinsenturk Üşeniyorum! Öyleyse, boşver... '
Metin : 0
Şentürk : 0
Twitterda : 0
Üşeniyorum : 0
Öyleyse : 0
boşver : 0


TWEET: ' Frozen Girls Anna and Elsa Backpack Only $9.99 (Reg $19.99) http://t.co/N2qucmI008 #onlinedeals #deal #onlinedeal '
Frozen : 0
Girls : 0
Anna : 0
and : 0
Elsa : 0
Backpack : 0
Only : 0
Reg : 0


TWEET: ' @duygu_damar direnemedin instagrama

متكبرا : 0
فاعلم : 0
أنه : 0
قليل : 0
الصلاة : 0
أو : 0
عديمها : 0
لا : 0
يجتمع : 0
كبر : 0
مع : 0
كثرة : 0
سجود : 0
سيماهم : 0
في : 0
وجوههم : 0
من : 0
أثر : 0
السجود : 0
صح : 0
عن : 0
مجاهد : 0


TWEET: ' header ? 👀 '
header : 0


TWEET: ' Después de 2 meses de que las entradas salieran a la venta, así va el #MalúCuartoPalacio @_MaluOficial_ 💃💃💃💃 http://t.co/boNW9zMZc4 '
Después : 0
de : 0
meses : 0
de : 0
que : 0
las : 0
entradas : 0
salieran : 0
a : 0
la : 0
venta : 0
así : 0
va : 0
el : 0


TWEET: ' Aula só 15:15 agora '
Aula : 0
só : 0
agora : 0


TWEET: ' Meryem ile bir dedikodu yaptık varya anlatamam :Dd 
 Metin Şentürk Twitterda @metinsenturk '
Meryem : 0
ile : 0
bir : 0
dedikodu : 0
yaptık : 0
varya : 0
anlatamam : 0
Dd : 0
Metin : 0
Şentürk : 0
Twitterda : 0


TWEET: ' nasıl olurda???Hâla bıkmadığım şarkı tadındasın.. '
nasıl : 0
olurda : 0
Hâla : 0
bıkmadığım : 0
şarkı : 0
tadındasın : 0


TWEET: ' Hadi artık sen konuş, herkes sussun.. 
 Metin Şentürk Twitterda @metinsentur

상위 : 0
중 : 0
은 : 0
구상금 : 0
한푼 : 0
안 : 0
내 : 0
gt : 0
건강보험 : 0
구상금 : 0
미납자 : 0
중 : 0
고액자산가 : 0
상위 : 0
명은 : 0
억 : 0
억원대의 : 0
부자 : 0
이들의 : 0
구상금 : 0
미징수율 : 0
대로 : 0
전체 : 0
평균의 : 0
배 : 0
매해 : 0
억원의 : 0
재정누수 : 0


TWEET: ' http://t.co/lbtoxOp5Gn - Tα ονόματα στη ΔΗΚΕΒΟ και στη ΔΕΥΑΒ :: Σιάτιστα http://t.co/ROqObR7CUU via @webnode '
Tα : 0
ονόματα : 0
στη : 0
ΔΗΚΕΒΟ : 0
και : 0
στη : 0
ΔΕΥΑΒ : 0
Σιάτιστα : 0
via : 0


TWEET: ' RT @NKsubs: Leeteuk: Yeaaa (gets up)
Kangin: Hyung, sit down. Really
Heechul: Don't be like that to my friend! '
Leeteuk : 2
Yeaaa : 2
gets : 2
up : 2
Kangin : 2
Hyung : 2
sit : 2
down : 2
Really : 2
Heechul : 2
Don't : 2
be : 2
that : 2
to : 2
my : 2
friend : 2


TWEET: ' 【自動】おやすみるきーヾ(･ω･o) '
自動 : 0
おやすみるきーヾ : 0
ω : 0
o : 0


TWEET: ' RT @Gabriele_Corno: Natural History Museum @ London by Richard Beresford Harris #architecture #Prospective http://t.co/cWiBisJQDr '
History : 1
Museum : 1
London : 1
by : 1
Richard : 1
Beresford : 1
Harris : 1


TWEET: ' @_mjodcx تمم '
ت